In [ ]:
import numpy as np
import en_core_web_sm
from gensim.models import Phrases
import gensim
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [ ]:
newArray = np.load('c:\\Users\\psamp\\AnacondaProjects\\PdfData\\corpus.npy')

In [ ]:
newArray1 = np.load('c:\\Users\\psamp\\AnacondaProjects\\PdfData\\y_train.npy')

In [ ]:
#LabelArray = newArray1[20:]
LabelArray = newArray1

In [ ]:
#convert numpy to list
sampledoc = newArray.tolist()

In [ ]:
#remove new line
doc = []
for w in sampledoc :
    doc.append(w.replace('\n',' '))
    

In [ ]:
newdoc = []
#for w in doc[20:]:
for w in doc:
    newdoc.append(w[0:w.find('C E R T')])
 

In [ ]:
nlp = en_core_web_sm.load()

In [ ]:
my_stop_words = [u'thanks',u'Mr',u'mr',u'Mrs',u'mrs',u'Mr.',u'Mrs.',u'Ms',u'Ms.',u'ms',u'To',u'the',u'The',u'there',u'There',u' ',u'$',u'\'s',u' Q ',u' Q. ',u' A ',u' A. '
                ,u'q.',u'a.',u'docket_no.',u'docket_no',u'witness',u'cross',u'a',u'q',u'I',u'You',u'you',u'They',u'they',u'me',u'He',u'he',u'Her',u'her',u'him',u'She',u'she'
                ,u'We',u'we',u'us',u'They',u'them']
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [ ]:
notes,details = [],[] 
for x in range(len(newdoc)):
    newdoc_l = nlp(str(newdoc[x]))  
    for w in newdoc_l:        
        if not w.is_stop and not w.is_punct and not w.like_num and not w.is_space:
            details.append(w.lemma_)
    notes.append(details)
    details = []

In [ ]:
notes[40]

In [ ]:
bigram = Phrases(notes)

In [ ]:
notes = [bigram[line] for line in notes]

In [ ]:
len(notes)

In [ ]:
bigram = Phrases(notes)

In [ ]:
notes = [bigram[line] for line in notes]

In [ ]:
notes

In [ ]:
t= Tokenizer() 

In [ ]:
#text preprocessing in keras
finalnotes = []    
for w in notes:
    #combinedString = string.join(w," ")
    finalnotes.append(' '.join(w))

In [ ]:
len(finalnotes)

In [ ]:
t.fit_on_texts(finalnotes)

In [ ]:
sequences = t.texts_to_sequences(finalnotes)

In [ ]:
sequences

In [ ]:
wordcount = []

for w in finalnotes:
    wordcount.append(len(re.findall("\S+",w)))

In [ ]:
wordcount

In [ ]:
sequences = sequence.pad_sequences(sequences,maxlen= 20000)

In [ ]:
y_train = sequence.pad_sequences(LabelArray,maxlen=18)

In [ ]:
y_train

In [ ]:
doc2vec = Word2Vec(finalnotes,size=50, window=7, sg=1 )

In [ ]:
embedding_matrix = np.zeros((len(t.word_index) + 1,50))
for word,i in t.word_index.items():
    try : 
        embedding_vector = doc2vec[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        pass
print(embedding_matrix.shape)    

In [ ]:
 # input dim - total word in vocabulary
 # output dim - size of the vector typically 100 - 300    
 # input length - max length   
 model = Sequential()
 model.add(Embedding(input_dim=len(t.word_index)+ 1, 
                       output_dim=50 ,               
                       weights=[embedding_matrix],
                       input_length=20000,           
                       mask_zero=True,
                       trainable=False ))   
 model.add(LSTM(75))   
 model.add(Dropout(0.3)) 
 model.add(Dense(18,activation='sigmoid'))   

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(t.word_index)+ 1,output_dim=50 ,                       
                     input_length=30000,
                     mask_zero=True,
                     trainable=False ))   
model.add(LSTM(50))   
model.add(Dropout(0.3)) 
model.add(Dense(18,activation='sigmoid'))   

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(t.word_index)+ 1,
                    output_dim=50 , 
                    weights=[embedding_matrix],
                    input_length=30000,
                    mask_zero=True,
                     ))   
model.add(SimpleRNN(100))   
model.add(Dropout(0.1)) 
model.add(Dense(18,activation='sigmoid'))   

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(t.word_index)+ 1,
                    output_dim=50 , 
                    input_length=20000,
                    mask_zero=True,
                     ))   
model.add(SimpleRNN(100))   
model.add(Dropout(0.1)) 
model.add(Dense(18,activation='sigmoid'))   

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(10000,)))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(18, activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(sequences, y_train, validation_split=0.20, epochs=50, batch_size=1)
